## State Tax Revenue Across the U.S.

(Insert some informative text here)

In [1]:
list.files(pattern = "*.csv")

# From https://taxfoundation.org/sources-state-and-local-tax-revenues/
tax_type_percents <- read.csv("2010_state_tax_breakdown.csv", stringsAsFactors = FALSE, sep = "|")
head(tax_type_percents, 10)

# From https://www.taxadmin.org/2018-state-tax-revenue
# total_tax in millions-$'s, per_Capita in $-s
tax_revenues <- read.csv("2018_state_tax_revenues.csv", stringsAsFactors = FALSE, sep = "|")
head(tax_revenues, 10)

# Compare the length of the 2 data sets
nrow(tax_type_percents)
nrow(tax_revenues)

[1] "2010_state_tax_breakdown.csv" "2018_state_tax_revenues.csv"

,State,Property_Tax,Sales_Tax,Indiv_Income_Tax,Corp_Income_Tax,Other_Taxes
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alabama,19.4,47.5,20.3,3.2,9.6
2,Alaska,21.4,10.9,NA,10.4,57.3
3,Arizona,37.3,44.4,12.3,2.1,4.0
4,Arkansas,18.3,50.5,22.0,4.1,5.1
5,California,31.2,30.1,26.4,5.3,7.0
6,Colorado,39.1,32.9,20.0,1.8,6.2
7,Connecticut,42.0,25.1,26.9,2.4,3.6
8,Delaware,18.6,13.2,25.3,4.1,38.8
9,Florida,42.9,46.9,NA,2.7,7.4


,State,Total_Taxes,Per_Capita,Rank,Perc_of_pers_Income,Rank.1
,<chr>,<int>,<int>,<int>,<dbl>,<int>
1,Alabama,11056,2262,42,5.6,34
2,Alaska,1642,2226,44,3.9,50
3,Arizona,16294,2272,41,5.5,38
4,Arkansas,9843,3266,17,8.0,7
5,California,175017,4424,8,7.4,10
6,Colorado,14803,2599,36,4.8,44
7,Connecticut,19081,5341,4,7.4,9
8,Delaware,4220,4363,9,8.8,5
9,Dist_of_Columbia,7959,11331,1,14.3,1


[1] 51

[1] 51

## Data Cleanup/Reshape

### To Do:
- [X] Check to see if the state names match between each data set.
- [X] Fix the non-matching state names once they're matched.
- [X] Drop the rank columns from revenues table
- [X] Join the 2 data sets together by state name
- [X] Convert total_tax from "millions" into tax_revenue in "dollars"
- [X] Create population column based on per_capita tax amount
- [ ] Convert tax percentages into actual dollar amounts
- [ ] Convert the "Perc_pers_Income" into dollar amounts

### Check state names for non-matches and update bad records

In [2]:
# Determine the difference between the names of each data set
df1_names <- tax_type_percents$State
df2_names <- tax_revenues$State

# setdiff(X, Y) returns the entries in X that don't match what's in Y
print(setdiff(df1_names, df2_names))
# 'West_Virginia' 'District_of_Columbia'

print(setdiff(df2_names, df1_names))
# "Dist_of_Columbia" "W_Virginia"      

# grep() returns the indices for the match pattern
grep("Columbia", df2_names)
grep("_Virginia", df2_names)

# which() returns the indices for the matching condition
which(df2_names == "Dist_of_Columbia")
which(df2_names == "W_Virginia")

# Replace the names in 'tax_revenues' that are abbreviated to match those in 'tax_type_percent'

# we can use index position from df2_names because it matches tax_revenues$State column
tax_revenues$State[grep("Columbia", df2_names)] <- "District_of_Columbia"
tax_revenues$State[grep("_Virginia", df2_names)] <- "West_Virginia"

[1] "West_Virginia"        "District_of_Columbia"
[1] "Dist_of_Columbia" "W_Virginia"      


[1] 9

[1] 49

[1] 9

[1] 49

In [3]:
# So what's the difference between using 'grep' and 'which'?
# When used in this way, they're the same
grep(2, c(10,9,8,7,6,5,4,3,2,1))
which(c(10,9,8,7,6,5,4,3,2,1) == 2)

# However 'which' provides greater flexibility when checking for a condition
which(c(10,9,8,7,6,5,4,3,2,1) <= 3)

[1] 9

[1] 9

[1]  8  9 10

In [4]:
# Check that the columns match now
print(setdiff(tax_type_percents$State, tax_revenues$State))

character(0)


### Drop the rank columns from 'tax_revenues'

NOTE: When using 'select' this way, DO NOT put the names of the columns in quotes.  

This will not work! --> `tax_revenues <- subset(tax_revenues, select = -c("Rank", "Rank.1"))`

In [5]:
names(tax_revenues)
tax_revenues <- subset(tax_revenues, select = -c(Rank, Rank.1))
names(tax_revenues)

[1] "State"               "Total_Taxes"         "Per_Capita"         
[4] "Rank"                "Perc_of_pers_Income" "Rank.1"

[1] "State"               "Total_Taxes"         "Per_Capita"         
[4] "Perc_of_pers_Income"

### Join 2 data sets by column name

Use the `merge()` command.

In [6]:
tax_data <- merge(tax_revenues, tax_type_percents, by = "State")
head(tax_data)

,State,Total_Taxes,Per_Capita,Perc_of_pers_Income,Property_Tax,Sales_Tax,Indiv_Income_Tax,Corp_Income_Tax,Other_Taxes
,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alabama,11056,2262,5.6,19.4,47.5,20.3,3.2,9.6
2,Alaska,1642,2226,3.9,21.4,10.9,NA,10.4,57.3
3,Arizona,16294,2272,5.5,37.3,44.4,12.3,2.1,4.0
4,Arkansas,9843,3266,8.0,18.3,50.5,22.0,4.1,5.1
5,California,175017,4424,7.4,31.2,30.1,26.4,5.3,7.0
6,Colorado,14803,2599,4.8,39.1,32.9,20.0,1.8,6.2


### Convert 'Total_Taxes' from "millions" into 'tax_revenue' in "dollars"

In [7]:
tax_data$tax_revenue <- (tax_data$Total_Taxes * 1000000)
head(tax_data)

,State,Total_Taxes,Per_Capita,Perc_of_pers_Income,Property_Tax,Sales_Tax,Indiv_Income_Tax,Corp_Income_Tax,Other_Taxes,tax_revenue
,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alabama,11056,2262,5.6,19.4,47.5,20.3,3.2,9.6,1.10560e+10
2,Alaska,1642,2226,3.9,21.4,10.9,NA,10.4,57.3,1.64200e+09
3,Arizona,16294,2272,5.5,37.3,44.4,12.3,2.1,4.0,1.62940e+10
4,Arkansas,9843,3266,8.0,18.3,50.5,22.0,4.1,5.1,9.84300e+09
5,California,175017,4424,7.4,31.2,30.1,26.4,5.3,7.0,1.75017e+11
6,Colorado,14803,2599,4.8,39.1,32.9,20.0,1.8,6.2,1.48030e+10


### Create 'population' column from 'Per_Capita' and 'tax_revenue' columns

In [10]:
tax_data$population <- as.integer(tax_data$tax_revenue / tax_data$Per_Capita)
tax_data$Total_Taxes <- NULL

# Rename column
tax_data$income_perc <- tax_data$Perc_of_pers_Income
tax_data$Perc_of_pers_Income <- NULL


head(tax_data)

,State,Per_Capita,Property_Tax,Sales_Tax,Indiv_Income_Tax,Corp_Income_Tax,Other_Taxes,tax_revenue,population,income_perc
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,Alabama,2262,19.4,47.5,20.3,3.2,9.6,1.10560e+10,4887709,5.6
2,Alaska,2226,21.4,10.9,NA,10.4,57.3,1.64200e+09,737646,3.9
3,Arizona,2272,37.3,44.4,12.3,2.1,4.0,1.62940e+10,7171654,5.5
4,Arkansas,3266,18.3,50.5,22.0,4.1,5.1,9.84300e+09,3013778,8.0
5,California,4424,31.2,30.1,26.4,5.3,7.0,1.75017e+11,39560804,7.4
6,Colorado,2599,39.1,32.9,20.0,1.8,6.2,1.48030e+10,5695652,4.8


In [13]:
## Reverse engineer per_capita_income
tax_data$per_cap_income <- tax_data$Per_Capita / (tax_data$income_perc / 100)
tax_data

State,Per_Capita,Property_Tax,Sales_Tax,Indiv_Income_Tax,Corp_Income_Tax,Other_Taxes,tax_revenue,population,income_perc,per_cap_income
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
Alabama,2262,19.4,47.5,20.3,3.2,9.6,1.10560e+10,4887709,5.6,40392.86
Alaska,2226,21.4,10.9,NA,10.4,57.3,1.64200e+09,737646,3.9,57076.92
Arizona,2272,37.3,44.4,12.3,2.1,4.0,1.62940e+10,7171654,5.5,41309.09
Arkansas,3266,18.3,50.5,22.0,4.1,5.1,9.84300e+09,3013778,8.0,40825.00
California,4424,31.2,30.1,26.4,5.3,7.0,1.75017e+11,39560804,7.4,59783.78
Colorado,2599,39.1,32.9,20.0,1.8,6.2,1.48030e+10,5695652,4.8,54145.83
Connecticut,5341,42.0,25.1,26.9,2.4,3.6,1.90810e+10,3572551,7.4,72175.68
Delaware,4363,18.6,13.2,25.3,4.1,38.8,4.22000e+09,967224,8.8,49579.55
District_of_Columbia,11331,37.0,27.0,22.0,6.5,7.5,7.95900e+09,702409,14.3,79237.76
